In this notebook we will explain the RIPPL workflow based on a study case about the earthquake on one of the islands
of Hawaii on May 4th 2018

In the first block we define the study area. There are different options to do so.
1. Create a shapefile (this can be done using ArcGIS or QGIS software)
2. Create a kml file using google earth > https://www.google.com/earth/
3. Create a geojson using > http://geojson.io
4. Create a list of coordinates in lat/lon coordinate pairs. This is what we will do here. 

To show the study area it is possible to run both google earth and geojson within the notebook.

For further background on this event you can read:
https://en.wikipedia.org/wiki/2018_Hawaii_earthquake

Following image shows the earthquake strength (credits: USGS 2018):
![Image of Hawaii earthquake strength](https://en.wikipedia.org/wiki/2018_Hawaii_earthquake#/media/File:2018_Hawaii_earthquake.jpg)

In [1]:
# Import script to embed geolocation.
from IPython.display import HTML
import datetime
import sys
import shutil
import numpy as np
import os
sys.path.extend(['/Users/gertmulder/software/rippl_main'])

import rippl

from rippl.orbit_geometry.read_write_shapes import ReadWriteShapes
from rippl.SAR_sensors.sentinel.sentinel_download import DownloadSentinel
from rippl.processing_templates.general_sentinel_1 import GeneralPipelines

France_shape = [(7.20, 43.75), (7.50, 43.75), (7.50, 43.90), (7.20, 43.90), (7.20, 43.75)]
study_area = ReadWriteShapes()
study_area(France_shape)

geojson = study_area.shape

# Try to do the same by creating a shapefile with QGIS, geojson online or a .kml file in google earth.
# study_area.read_kml(kml_path)
# study_area.read_geo_json(geojson_path)
# study_area.read_shapefile(shapefile_path)

After selection of the right track we can start the actual download of the images. In our case we use track 88.

This will download our data automatically to our radar database. Additionally, it will download the precise orbit files.
These files are created within a few weeks after the data acquisition and define the satellite orbit within a few cm
accuracy. These orbits are necessary to accurately define the positions of the radar pixels on the ground later on
in the processing.

In [2]:
# Track and data type of Sentinel data
mode = 'IW'
product_type = 'SLC'
polarisation = 'VV'

from rippl.processing_templates.general_sentinel_1 import GeneralPipelines

# Create the list of the 4 different stacks.
track_no = 88
stack_name = 'France_old_data'
tmp_directory = '/Users/gertmulder/SAR_tmp'
if not os.path.exists(tmp_directory):
    os.mkdir(tmp_directory)
coreg_tmp_directory = '/Users/gertmulder/SAR_tmp'
if not os.path.exists(coreg_tmp_directory):
    os.mkdir(coreg_tmp_directory)
ml_grid_tmp_directory = '/Users/gertmulder/SAR_tmp'
if not os.path.exists(ml_grid_tmp_directory):
    os.mkdir(ml_grid_tmp_directory)

# For every track we have to select a master date. This is based on the search results earlier.
# Choose the date with the lowest coverage to create an image with only the overlapping parts.
start_date = datetime.datetime(year=2016, month=3, day=3)
end_date = datetime.datetime(year=2016, month=4, day=18)
master_date = datetime.datetime(year=2016, month=3, day=22)

# Number of processes for parallel processing. Make sure that for every process at least 2GB of RAM is available
no_processes = 4

s1_processing = GeneralPipelines(processes=no_processes)
s1_processing.download_sentinel_data(start_date=start_date, end_date=end_date, track=track_no,
                                           polarisation=polarisation, shapefile=study_area.shape, data=True, source='ASF')
s1_processing.create_sentinel_stack(start_date=start_date, end_date=end_date, master_date=master_date, cores=no_processes,
                                          track=track_no,stack_name=stack_name, polarisation=polarisation,
                                          shapefile=study_area.shape, mode=mode, product_type=product_type)

# Finally load the stack itself. If you want to skip the download step later, run this line before other steps!
s1_processing.read_stack(start_date=start_date, end_date=end_date, stack_name=stack_name)

Loading available orbit data files
Finished loading date files
S1A_IW_SLC__1SDV_20160310T172230_20160310T172257_010310_00F422_204E.zip
S1A_IW_SLC__1SDV_20160310T172205_20160310T172232_010310_00F422_CB7A.zip
S1A_IW_SLC__1SDV_20160403T172231_20160403T172258_010660_00FE19_7395.zip
S1A_IW_SLC__1SDV_20160403T172206_20160403T172233_010660_00FE19_79EB.zip
S1A_IW_SLC__1SDV_20160322T172219_20160322T172246_010485_00F908_2AD9.zip
S1A_IW_SLC__1SDV_20160415T172214_20160415T172241_010835_01035A_07FA.zip
Selected images:
S1A_IW_SLC__1SDV_20160310T172230_20160310T172257_010310_00F422_204E.zip
S1A_IW_SLC__1SDV_20160310T172205_20160310T172232_010310_00F422_CB7A.zip
S1A_IW_SLC__1SDV_20160403T172231_20160403T172258_010660_00FE19_7395.zip
S1A_IW_SLC__1SDV_20160403T172206_20160403T172233_010660_00FE19_79EB.zip
S1A_IW_SLC__1SDV_20160322T172219_20160322T172246_010485_00F908_2AD9.zip
S1A_IW_SLC__1SDV_20160415T172214_20160415T172241_010835_01035A_07FA.zip

Reading precise and restituted orbit database
Reading s

To define the location of the radar pixels on the ground we need the terrain elevation. Although it is possible to 
derive terrain elevation from InSAR data, our used Sentinel-1 dataset is not suitable for this purpose. Therefore, we
download data from an external source to create a digital elevation model (DEM). In our case we use SRTM data. 

However, to find the elevation of the SAR data grid, we have to resample the data to the radar grid first to make it
usable. This is done in the next steps.

In [3]:
# Some basic settings for DEM creation.
dem_buffer = 0.1        # Buffer around radar image where DEM data is downloaded
dem_rounding = 0.1      # Rounding of DEM size in degrees
dem_type = 'SRTM1'      # DEM type of data we download (SRTM1, SRTM3 and TanDEM-X are supported)

# Define both the coordinate system of the full radar image and imported DEM
s1_processing.create_radar_coordinates()
s1_processing.create_dem_coordinates(dem_type=dem_type)

# Download external DEM
s1_processing.download_external_dem(dem_type=dem_type, buffer=dem_buffer, rounding=dem_rounding, n_processes=no_processes)

Using the obtained elevation model the exact location of the radar pixels in cartesian (X,Y,Z) and geographic (Lat/Lon)
can be derived. This is only done for the master or reference image. This process is referred to as geocoding.

In [4]:
# Geocoding of image
s1_processing.geocoding()

Processing pipeline block 1 out of 1
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Processing pipeline block 1 out of 1
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Concatenated dataset already exists. If you want to overwrite set overwrite to True
Concatenated dataset already exists. If you want to overwrite set overwrite to True
Concatenated dataset already exists. If you want to overwrite set overwrite to True
Concatenated dataset already exists. If you want to overwrite set overwrite to True
Concatenated dataset already exists. If you want to overwrite set overwrite to True


The information from the geocoding can directly be used to find the location of the master grid pixels in the slave
grid images. This process is called coregistration. Because the orbits are not exactly the same with every satellite 
overpass but differ hundreds to a few thousand meters every overpass, the grids are slightly shifted with respect to 
each other. These shift are referred to as the spatial baseline of the images. To correctly overlay the master and slave
images the software coregisters and resamples to the master grid.

To do so the following steps are done:
1. Coregistration of slave to master image
2. Deramping the doppler effects due to TOPs mode of Sentinel-1 satellite
3. Resampling of slave image
4. Reramping resampled slave image.

Due to the different orbits of the master and slave image, the phase of the radar signal is also shifted. We do not 
know the exact shift of the two image, but using the geometry of the two images we can estimate the shift of the phase
between different pixels. Often this shift is split in two contributions:
1. The flat earth phase. This phase is the shift in the case the earth was a perfect ellipsoid
2. The topographic phase. This is the phase shift due to the topography on the ground.
In our processing these two corrections are done in one go.

In [8]:
# Next step applies resampling and phase correction in one step.
# Polarisation

# Because with the geometric coregistrtation we load the X,Y,Z files of the main image for every calculation it can
# be beneficial to load them to a fast temporary disk. (If enough space you should load them to memory disk)
s1_processing.geometric_coregistration_resampling(polarisation=polarisation, output_phase_correction=True,
                                                  coreg_tmp_directory=coreg_tmp_directory, tmp_directory=tmp_directory, baselines=False, height_to_phase=True)
shutil.rmtree(coreg_tmp_directory)
os.mkdir(coreg_tmp_directory)
shutil.rmtree(tmp_directory)
os.mkdir(tmp_directory)

Processing pipeline block 1 out of 1
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished
Skipping processing. Process already finished


Now we can create calibrated amplitudes, interferograms and coherences.

In [6]:

# Load the images in blocks to temporary disk (or not if only coreg data is loaded to temp disk)
temporal_baseline = 30
min_timespan = temporal_baseline * 2
# Every process can only run 1 multilooking job. Therefore, in the case of amplitude calculation the number of processes
# is limited too the number of images loaded.
amp_processing_efficiency = 0.5
effective_timespan = np.maximum(no_processes * 6 * amp_processing_efficiency, min_timespan)

no_days = datetime.timedelta(days=int(effective_timespan / 2))
if no_days < (end_date - start_date):
    step_date = start_date
    step_dates = []
    while step_date < end_date:
        step_dates.append(step_date)
        step_date += no_days
    step_dates.append(end_date)

    start_dates = step_dates[:-2]
    end_dates = step_dates[2:]
else:
    end_dates = [end_date]
    start_dates = [start_date]


if not isinstance(polarisation, list):
    pol = [polarisation]
else:
    pol = polarisation

for start_date, end_date in zip(start_dates, end_dates):
    s1_processing.read_stack(start_date=start_date, end_date=end_date, stack_name=stack_name)
    # We split the different polarisation to limit the number of files in the temporary folder.
    for p in pol:
        for dist in [50, 100, 200]:
            # The actual creation of the calibrated amplitude images
            s1_processing.create_ml_coordinates(standard_type='oblique_mercator', dx=dist, dy=dist, buffer=0, rounding=0)
            s1_processing.prepare_multilooking_grid(polarisation)
            s1_processing.create_calibrated_amplitude_multilooked(polarisation, coreg_tmp_directory=coreg_tmp_directory, tmp_directory=tmp_directory)
            s1_processing.create_output_tiffs_amplitude()

            s1_processing.create_ifg_network(temporal_baseline=temporal_baseline)
            s1_processing.create_interferogram_multilooked(polarisation, coreg_tmp_directory=ml_grid_tmp_directory, tmp_directory=tmp_directory)
            s1_processing.create_coherence_multilooked(polarisation, coreg_tmp_directory=ml_grid_tmp_directory, tmp_directory=tmp_directory)

            # Create output geotiffs
            s1_processing.create_output_tiffs_coherence_ifg()

            # Create lat/lon/incidence angle/DEM for multilooked grid.
            s1_processing.create_geometry_mulitlooked(baselines=True, height_to_phase=True)
            s1_processing.create_output_tiffs_geometry()

            # The coreg temp directory will only contain the loaded input lines/pixels to do the multilooking. These
            # files will be called by every process so it can be usefull to load them in memory the whole time.
            # If not given, these files will be loaded in the regular tmp folder.
            if coreg_tmp_directory:
                if os.path.exists(ml_grid_tmp_directory):
                    shutil.rmtree(ml_grid_tmp_directory)
                    os.mkdir(ml_grid_tmp_directory)

        for dist in [50, 100, 200]:
            # The actual creation of the calibrated amplitude images
            s1_processing.create_ml_coordinates(standard_type='UTM', dx=dist, dy=dist, buffer=0, rounding=0)
            s1_processing.prepare_multilooking_grid(polarisation)
            s1_processing.create_calibrated_amplitude_multilooked(polarisation, coreg_tmp_directory=coreg_tmp_directory, tmp_directory=tmp_directory)
            s1_processing.create_output_tiffs_amplitude()

            s1_processing.create_ifg_network(temporal_baseline=temporal_baseline)
            s1_processing.create_interferogram_multilooked(polarisation, coreg_tmp_directory=ml_grid_tmp_directory, tmp_directory=tmp_directory)
            s1_processing.create_coherence_multilooked(polarisation, coreg_tmp_directory=ml_grid_tmp_directory, tmp_directory=tmp_directory)

            # Create output geotiffs
            s1_processing.create_output_tiffs_coherence_ifg()

            # Create lat/lon/incidence angle/DEM for multilooked grid.
            s1_processing.create_geometry_mulitlooked(baselines=True, height_to_phase=True)
            s1_processing.create_output_tiffs_geometry()

            # The coreg temp directory will only contain the loaded input lines/pixels to do the multilooking. These
            # files will be called by every process so it can be usefull to load them in memory the whole time.
            # If not given, these files will be loaded in the regular tmp folder.
            if coreg_tmp_directory:
                if os.path.exists(ml_grid_tmp_directory):
                    shutil.rmtree(ml_grid_tmp_directory)
                    os.mkdir(ml_grid_tmp_directory)

        for dist in [0.0005, 0.001, 0.002]:
            # The actual creation of the calibrated amplitude images
            s1_processing.create_ml_coordinates(dlat=dist, dlon=dist, coor_type='geographic', buffer=0, rounding=0)
            s1_processing.prepare_multilooking_grid(polarisation)
            s1_processing.create_calibrated_amplitude_multilooked(polarisation, coreg_tmp_directory=coreg_tmp_directory, tmp_directory=tmp_directory)
            s1_processing.create_output_tiffs_amplitude()

            s1_processing.create_ifg_network(temporal_baseline=temporal_baseline)
            s1_processing.create_interferogram_multilooked(polarisation, coreg_tmp_directory=ml_grid_tmp_directory, tmp_directory=tmp_directory)
            s1_processing.create_coherence_multilooked(polarisation, coreg_tmp_directory=ml_grid_tmp_directory, tmp_directory=tmp_directory)

            # Create output geotiffs
            s1_processing.create_output_tiffs_coherence_ifg()

            # Create lat/lon/incidence angle/DEM for multilooked grid.
            s1_processing.create_geometry_mulitlooked(baselines=True, height_to_phase=True)
            s1_processing.create_output_tiffs_geometry()

            # The coreg temp directory will only contain the loaded input lines/pixels to do the multilooking. These
            # files will be called by every process so it can be usefull to load them in memory the whole time.
            # If not given, these files will be loaded in the regular tmp folder.
            if coreg_tmp_directory:
                if os.path.exists(ml_grid_tmp_directory):
                    shutil.rmtree(ml_grid_tmp_directory)
                    os.mkdir(ml_grid_tmp_directory)

        if tmp_directory:
            if os.path.exists(tmp_directory):
                shutil.rmtree(tmp_directory)
                os.mkdir(tmp_directory)

Using predefined grid size for concatenation of slices
Concatenated dataset already exists. If you want to overwrite set overwrite to True
Processing pipeline block 1 out of 1
Processing pipeline block 1 out of 1


BrokenPipeError: Pipeline processing for /Users/gertmulder/SAR_processing/radar_data_stacks/sentinel1/France_old_data/20160310 failed.